In [1]:
# 版权声明：本文为CSDN博主「卡农2020」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
# 原文链接：https://blog.csdn.net/u012034742/article/details/106318906
import cv2
from functools import reduce
from PIL import Image
from sklearn.cluster import KMeans
from collections import Counter
from matplotlib import pyplot as plt
import cv2
import os
from os.path import basename
import json
import operator

In [2]:

# 计算两个图片相似度函数ORB算法
def ORB_img_similarity(img1_path,img2_path):
    """
    :param img1_path: 图片1路径
    :param img2_path: 图片2路径
    :return: 图片相似度
    """
    try:
        # 读取图片
        img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)
 
        # 初始化ORB检测器
        orb = cv2.ORB_create()
        kp1, des1 = orb.detectAndCompute(img1, None)
        kp2, des2 = orb.detectAndCompute(img2, None)
 
        # 提取并计算特征点
        bf = cv2.BFMatcher(cv2.NORM_HAMMING)
        # knn筛选结果
        matches = bf.knnMatch(des1, trainDescriptors=des2, k=2)
 
        # 查看最大匹配点数目
        good = [m for (m, n) in matches if m.distance < 0.75 * n.distance]
        similary = len(good) / len(matches)
        return similary
 
    except:
        return '0'

In [3]:

# 计算图片的局部哈希值--pHash
def phash(img):
    """
    :param img: 图片
    :return: 返回图片的局部hash值
    """
    img = img.resize((8, 8), Image.ANTIALIAS).convert('L')
    avg = reduce(lambda x, y: x + y, img.getdata()) / 64.
    hash_value=reduce(lambda x, y: x | (y[1] << y[0]), enumerate(map(lambda i: 0 if i < avg else 1, img.getdata())), 0)
    return hash_value


In [4]:
#计算两个图片相似度函数局部敏感哈希算法
def phash_img_similarity(img1_path,img2_path):
    """
    :param img1_path: 图片1路径
    :param img2_path: 图片2路径
    :return: 图片相似度
    """
    # 读取图片
    img1 = Image.open(img1_path)
    img2 = Image.open(img2_path)
   
 
    # 计算汉明距离
    distance = bin(phash(img1) ^ phash(img2)).count('1')
    similary = 1 - distance / max(len(bin(phash(img1))), len(bin(phash(img1))))
    return similary


In [5]:
# 直方图计算图片相似度算法
def make_regalur_image(img, size=(256, 256)):
    """我们有必要把所有的图片都统一到特别的规格，在这里我选择是的256x256的分辨率。"""
    return img.resize(size).convert('RGB')
 
def hist_similar(lh, rh):
    assert len(lh) == len(rh)
    return sum(1 - (0 if l == r else float(abs(l - r))/max(l, r)) for l, r in zip(lh, rh))/len(lh)
 
def calc_similar(li, ri):
    return sum(hist_similar(l.histogram(), r.histogram()) for l, r in zip(split_image(li), split_image(ri))) / 16.0
 
def calc_similar_by_path(lf, rf):
    li, ri = make_regalur_image(Image.open(lf)), make_regalur_image(Image.open(rf))
    return calc_similar(li, ri)
 
def split_image(img, part_size = (64, 64)):
    w, h = img.size
    pw, ph = part_size
    assert w % pw == h % ph == 0
    return [img.crop((i, j, i+pw, j+ph)).copy() for i in range(0, w, pw) \
            for j in range(0, h, ph)]



# new_similarity_compare

In [6]:
import os
import shutil


In [43]:
# 融合相似度阈值
threshold1 = 0.5
# 最终相似度较高判断阈值
threshold2 = 0.6


In [8]:

# 融合函数计算图片相似度
def calc_image_similarity(img1_path,img2_path):
    """
    :param img1_path: filepath+filename
    :param img2_path: filepath+filename
    :return: 图片最终相似度
    """
 
    similary_ORB=float(ORB_img_similarity(img1_path,img2_path))
    similary_phash=float(phash_img_similarity(img1_path,img2_path))
    similary_hist=float(calc_similar_by_path(img1_path, img2_path))
    # 如果三种算法的相似度最大的那个大于0.7，则相似度取最大，否则，取最小。
    max_three_similarity=max(similary_ORB,similary_phash,similary_hist)
    min_three_similarity=min(similary_ORB,similary_phash,similary_hist)
    print("ORB:"+str(similary_ORB))
    print("phash:"+str(similary_phash))
    print("hist:"+str(similary_hist))
    if max_three_similarity>threshold1:
        result=max_three_similarity
    else:
        result=min_three_similarity
    print("result:"+str(result))
 
    return round(result,3)

    

In [22]:
base_location = 'C:/Users/JIALI/Desktop/test/before/'
jList = os.listdir(base_location)
# filename = "000025"
def crop(base_location):
    
    for file in jList:
      #  crop_imgs = []
        if file.endswith(".jpg"):
            img = cv2.imread(os.path.join(base_location,file))
            #    plt.imshow(img)
        if file.endswith(".json"):
            with open(os.path.join(base_location,file) , 'r') as reader:
                jf = json.loads(reader.read())
            strjf = str(jf)
            maxi = strjf.count('item') + 1
            for i in range (1,maxi):
                stri = str(i)
                # 裁切區域的 x 與 y 座標（左上角）
                x1 = jf['item'+stri]['bounding_box'][0]
                y1 = jf['item'+stri]['bounding_box'][3]
                # 裁切區域的 x 與 y 座標（右下角）
                x2 = jf['item'+stri]['bounding_box'][2]
                y2 = jf['item'+stri]['bounding_box'][1]
                category_id = jf['item'+stri]['category_id']
                # 裁切圖片
                crop_img = img[y2:y1, x1:x2]
                #print("crop_img")
               # print(crop_img)
               # crop_img = cv2.cvtColor(crop_img,cv2.COLOR_BGR2RGB)
             #   crop_imgs.append(crop_img)
                
                name = file.split('.')[0]
                cv2.imwrite(base_location+'crop/'+name+'_'+str(category_id)+'.jpg', crop_img)

In [45]:
if __name__ == '__main__':
 
    # 搜索文件夹
    filepath = 'C:/Users/JIALI/Desktop/test/crop/' 
    crop(base_location)
    srcname =  "000189"
    category_id = "2"
    img1_path = filepath+srcname+"_"+category_id+".jpg"
    # 相似圖片存放路徑
    newfilepath = 'C:/Users/JIALI/Desktop/test/相似/'
 
    for parent, dirnames, filenames in os.walk(filepath):
        
        for filename in filenames:
            if category_id in filename.split('_')[1]:
                img2_path = filepath+filename
                # 相同非同一張圖片的則比較
                if img1_path != img2_path:
                    kk = calc_image_similarity(img1_path, img2_path)
                    try:
                        if kk >= threshold2:
                            similarname = filename.split('_')[0]
                            # 将两張照片同时copy到指定目錄
                            shutil.copy(base_location+srcname+".jpg", newfilepath)
                            shutil.copy(base_location+similarname+".jpg", newfilepath)
                    except Exception as e:
                        print(e)
                        pass

ORB:0.0
phash:0.6363636363636364
hist:0.22337748241659747
result:0.6363636363636364
ORB:0.005988023952095809
phash:0.5909090909090908
hist:0.34053673263105905
result:0.5909090909090908
ORB:0.0
phash:0.48484848484848486
hist:0.326150870493714
result:0.0
ORB:0.0029940119760479044
phash:0.3787878787878788
hist:0.3301888653849778
result:0.0029940119760479044
ORB:0.014970059880239521
phash:0.5
hist:0.3183190444901934
result:0.014970059880239521
ORB:0.011976047904191617
phash:0.3939393939393939
hist:0.34175634337697924
result:0.011976047904191617
ORB:0.008982035928143712
phash:0.6515151515151515
hist:0.4026982809940245
result:0.6515151515151515
ORB:0.020958083832335328
phash:0.7272727272727273
hist:0.4145482245820905
result:0.7272727272727273
